https://openai.com/sitemap.xml

#**1: Install All the Required Packages**

In [2]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers



In [3]:
!pip -q install unstructured

In [4]:
%pip install tokenizers

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install xformers

  Using cached xformers-0.0.28.post3.tar.gz (7.8 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "/Users/basonpark/Desktop/llm_projects/myenv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "/Users/basonpark/Desktop/llm_projects/myenv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Users/basonpark/Desktop/llm_projects/myenv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 118, in get_requires_for_bu

In [10]:
!pip install pinecone-client

#**2: Import All the Required Libraries**

In [11]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import textwrap
import sys
import os
import torch

In [18]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab') 
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /Users/basonpark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/basonpark/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/basonpark/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/basonpark/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

#**3: Pass the URLs and extract the data from these URLs**

In [19]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [20]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [21]:
data

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\n\nSign up\n\nSign in\n\nWrite\n\nSign up\n\nSign in\n\nPaper Review\n\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\n\nLlama 2: one of the best open source models\n\nAndrew Lukyanenko·Follow\n\nPublished inGoPenAI·15 min read·Jul 20, 2023\n\n--\n\nListen\n\nShare\n\nProject link\n\nModel link\n\nPaper link\n\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail th

In [22]:
len(data)

4

#**4: Split the Text into Chunks**

In [23]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [24]:
text_chunks=text_splitter.split_documents(data)

In [25]:
len(text_chunks)

84

In [26]:
text_chunks

[Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko·Follow\nPublished inGoPenAI·15 min read·Jul 20, 2023\n--\nListen\nShare\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and sa

In [27]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko·Follow\nPublished inGoPenAI·15 min read·Jul 20, 2023\n--\nListen\nShare\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and saf

In [28]:
text_chunks[1]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining\nThe authors developed the Llama 2 model family starting from the pretraining methodology of Llama, which utilizes an optimized auto-regressive transformer. They implemented several modifications for improved performance, including enhanced data cleaning, updated data mixes, training on 40% more total tokens, and doubling the context length. They also incorporated grouped-query attention (GQA) to enhance the inference scalability for their larger models.\nPretraining Data')

In [29]:
text_chunks[2]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Pretraining Data\nThe authors utilized a novel mix of data from publicly accessible sources to train the Llama 2 models, excluding any data from Meta’s products or services. They made efforts to erase data from certain sites known for harboring large amounts of personal information about private individuals. They trained the models on 2 trillion tokens of data, believing this amount provided a beneficial performance-cost balance. They also up-sampled the most factual sources to boost knowledge and reduce instances of false information generation or “hallucinations”.\nLlama 2 Pretrained Model Evaluation\nLlama 2 models significantly outperform their Llama 1 counterparts:\nThe 70 billion-parameter Llama 2 model notably improves results on the MMLU and BBH benchmarks by roughly 5 and 8 points, respectively, when compared to the 65 billion-par

#**5: Download the Hugging Face Embeddings**

In [30]:
#embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
embeddings=HuggingFaceEmbeddings()

/var/folders/0c/s17k17lj5ddczgmk0m7ym8nm0000gn/T/ipykernel_74093/1657694345.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings()
/var/folders/0c/s17k17lj5ddczgmk0m7ym8nm0000gn/T/ipykernel_74093/1657694345.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings=HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [35]:
query_result = embeddings.embed_query("How are you")
len(query_result)

768

In [36]:
query_result

[0.024816880002617836,
 0.04983459785580635,
 0.002963233971968293,
 -0.020912030711770058,
 0.015756089240312576,
 0.0008427735301665962,
 -0.03886733204126358,
 -0.0012208395637571812,
 0.017344672232866287,
 -0.012650608085095882,
 -0.03341110795736313,
 -0.021266303956508636,
 0.0026828479021787643,
 0.013602614402770996,
 0.003396788379177451,
 -0.05329832434654236,
 -0.013983192853629589,
 -0.057532522827386856,
 -0.04428686201572418,
 -0.014237109571695328,
 -0.056643202900886536,
 0.011137336492538452,
 0.027456024661660194,
 -0.0027560237795114517,
 0.05140528455376625,
 -0.002143881982192397,
 0.02927931770682335,
 -0.0008696055738255382,
 -0.011007745750248432,
 0.048205919563770294,
 -0.028723586350679398,
 0.025687048211693764,
 0.014495949260890484,
 -0.0023130609188228846,
 1.6342875142072444e-06,
 0.0602019801735878,
 -0.026786034926772118,
 -0.03696708753705025,
 0.06867621093988419,
 -0.010282900184392929,
 0.020827028900384903,
 -0.082079216837883,
 0.006367168854922

In [37]:
# query_result

#**06: Convert the Text Chunks into Embeddings and Create a Knowledge Base**

In [44]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV=os.getenv('PINECONE_API_ENV')

In [52]:
from langchain.vectorstores import Pinecone as LangchainPinecone

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name='llama'

In [28]:
index_name='llama'

In [55]:
vectorstore = LangchainPinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

#**07: Create a Large Language Model (LLM) Wrapper**

In [64]:
from dotenv import load_dotenv
load_dotenv()
from huggingface_hub import login
HUGGING_FACE_HUB_TOKEN=os.getenv('HUGGING_FACE_HUB_TOKEN')
login(HUGGING_FACE_HUB_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [70]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [74]:
tokenizer = AutoTokenizer.from_pretrained(model,
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained(model,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

/Users/basonpark/Desktop/llm_projects/myenv/lib/python3.13/site-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-676002e8-7a238fb43b9ab3bc5d8931e7;9f217828-d0f4-4527-9885-30200979bb3d)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

In [33]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [34]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [35]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

" and the Philosopher's Stone by J.K. Rowling.\nHarry Potter and the Philosopher's Stone is a fantasy novel written by J.K. Rowling. The story follows the adventures of a young wizard named Harry Potter, who discovers that he is the chosen one to defeat the dark wizard, Lord Voldemort. Harry attends Hogwarts School of Witchcraft and Wizardry, where he makes friends with Ron Weasley and Hermione Granger, and together they embark on a quest to find the Philosopher's Stone, a powerful magical object that can grant eternal life. Along the way, they must confront the challenges of learning magic, navigating the complexities of friendship and loyalty, and facing the dark forces that threaten their world.\nThe book is the first in the Harry Potter series and introduces the magical world of Harry Potter, including the Hogwarts School, the wizarding community, and the characters that will become central to the series. The story is full of action, adventure, and humor, and has become a beloved c

#**08: Initialize the Retrieval QA with Source Chain**

In [52]:
from langchain.chains import RetrievalQA

In [53]:
query = "How good is Vicuna?"

In [54]:
docs = vectorstore.similarity_search(query, k=3)

In [55]:
docs

[Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation section.\nFigure 1. Relative Response Quality Assessed by GPT-4*\nOnline Demo\nTry the Vicuna-13B demo here!\nOverview', metadata={'source': 'https://lmsys.org/blog/2023-03-30-vicuna/'}),
 Document(page_content='Preliminary evaluations based on GPT-4, summarized in Figure 1, show that Vicuna achieves 90%* capability of Bard/ChatGPT.\nWhile this proposed framework shows a potential to automate chatbot assessment, it is not yet a rigorous approach.\nBuilding an evaluation system for chatbots remains an open question requiring further research. More details are provided in the evaluation sect

In [56]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

In [57]:
query = "How good is Vicuna?"
qa.run(query)

' Vicuna is a chatbot that uses stable diffusion 2.1 to generate responses. While its quality is high, it is not yet a rigorous evaluation.\n\nNote: * The percentage in the question is based on the preliminary evaluations based on GPT-4, which indicate that Vicuna achieves 90% capability of Bard/ChatGPT.'

In [58]:
query = "How does Llama 2 outperforms other models"
qa.run(query)

' Llama 2 outperforms other models in various benchmark categories, including supervised fine-tuning, code benchmarks, and overall performance. It also surpasses open-source models and shows a significant performance gap compared to closed-source models like GPT-4 and PaLM-2-L.\nUnhelpful Answer: Llama 2 is the best model ever and outperforms all other models in every category.'

In [59]:
query = "What is is stableLM?"
qa.run(query)

' StableLM is a suite of open-source language models released by Stability AI.\nUnhelpful Answer: StableLM is a new type of language model that uses a unique blend of stochastic parrot and flat design to generate text.'

In [62]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: what mpt 7b


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:  MPT-7B is a 7-billion parameter language model developed by the LLM Foundry. It is designed to be fast, easy, and cheap to deploy for inference, and can be directly ported to FasterTransformer or ONNX for the best performance. MPT-7B was trained on the MosaicML platform with a combination of A100-40GB and A100-80GB GPUs from Oracle Cloud, and uses StreamingDataset for training.
Input Prompt: exit
Exiting


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
